<a href="https://colab.research.google.com/github/marioluciofjr/codesoiram.python/blob/main/probabilidadesbr2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import requests as rq
from scipy.stats import poisson
import warnings

In [ ]:
# Nomes de todos os times atuais da Série A do Brasileirão
escolhas = """
América Mineiro
Athletico Paranaense
Atlético Mineiro
Bahia
Botafogo
Corinthians
Coritiba
Cruzeiro
Cuiabá
Flamengo
Fluminense
Fortaleza
Goiás
Grêmio
Internacional
Palmeiras
Red Bull Bragantino
Santos
São Paulo
Vasco da Gama
"""

# Você vai escolher o mandante e o visitante para saber a probabilidade em seguida
mandante = input("Escolha um dos times como mandante: "+escolhas)
print()
visitante = input("Escolha um dos times como visitante: "+escolhas)

# Faz uma requisição de dados no site do Wikipedia e lê o HTML
pegarDados = rq.get("https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2023_-_S%C3%A9rie_A")
tabelas = pd.read_html(pegarDados.text)

# Extrai tanto a tabela de classificação quanto a de jogos atualizados
classificacao = tabelas[6]
jogos = tabelas[7]


# Relaciona nomes dos times com as siglas
nomes_times = list(jogos["Casa \ Fora"])
siglas = list(jogos.columns)
siglas.pop(0)
de_para_times = dict(zip(siglas, nomes_times))

# Ajusta a tabela para os confrontos ficarem na mesma linha
tabela_jogos_ajustada = jogos.set_index("Casa \ Fora")
tabela_jogos_ajustada = tabela_jogos_ajustada.unstack().reset_index()
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"level_0": "fora", "Casa \ Fora": "casa", 0: "resultado"})

# Função que mostre o nome do time para a sigla correspondente
def ajustar_sigla_time(linha):
    sigla = linha["fora"]
    nome = de_para_times[sigla]
    return nome

# Analisa linha a linha e edita a coluna fora
tabela_jogos_ajustada["fora"] = tabela_jogos_ajustada.apply(ajustar_sigla_time, axis=1)
tabela_jogos_ajustada = tabela_jogos_ajustada[tabela_jogos_ajustada["fora"]!=tabela_jogos_ajustada["casa"]]

# Resolve o NaN
tabela_jogos_ajustada["resultado"] = tabela_jogos_ajustada["resultado"].fillna("a jogar")
tabela_jogos_realizados = tabela_jogos_ajustada[tabela_jogos_ajustada["resultado"].str.contains("–")]

# Deleta a coluna Resultado dos jogos que faltam
tabela_jogos_faltantes = tabela_jogos_ajustada[~tabela_jogos_ajustada["resultado"].str.contains("–")]
tabela_jogos_faltantes = tabela_jogos_faltantes.drop(columns=["resultado"])

# Cria duas colunas para receber os gols dos mandantes e os gols dos visitantes
tabela_jogos_realizados[["gols_casa", "gols_fora"]] = tabela_jogos_realizados["resultado"].str.split("–", expand=True)
tabela_jogos_realizados = tabela_jogos_realizados.drop(columns=["resultado"])
tabela_jogos_realizados["gols_casa"] = tabela_jogos_realizados["gols_casa"].astype(int)
tabela_jogos_realizados["gols_fora"] = tabela_jogos_realizados["gols_fora"].astype(int)

# Calcula a média de gols feitos em casa e de gols sofridos em casa
media_gols_casa = tabela_jogos_realizados.groupby("casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols_feitos_casa", "gols_fora": "gols_sofridos_casa"})

# Calcula a média de gols feitos fora e de gols sofridos fora
media_gols_fora = tabela_jogos_realizados.groupby("fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols_sofridos_fora", "gols_fora": "gols_feitos_fora"})

# Mescla as duas tabelas de médias de gols
tabela_estatisticas = media_gols_casa.merge(media_gols_fora, left_index=True, right_index=True)
tabela_estatisticas = tabela_estatisticas.reset_index()
tabela_estatisticas = tabela_estatisticas.rename(columns={"casa": "time"})

# Estabele variáveis para o mandante e o visitante
time_casa = mandante
time_fora = visitante

# média de gols que o time mandante, jogando em casa, faz contra o time visitante, jogando fora de casa
lambda_casa = (tabela_estatisticas.loc[tabela_estatisticas["time"]==time_casa, "gols_feitos_casa"].iloc[0]
                * tabela_estatisticas.loc[tabela_estatisticas["time"]==time_fora, "gols_sofridos_fora"].iloc[0])



# média de gols que o time visitante faz fora de casa * media de gols que o time mandante sofre dentro de casa
lambda_fora =  (tabela_estatisticas.loc[tabela_estatisticas["time"]==time_fora, "gols_feitos_fora"].iloc[0]
                    * tabela_estatisticas.loc[tabela_estatisticas["time"]==time_casa, "gols_sofridos_casa"].iloc[0])

# Variáveis de vitória mandante, empate e vitória visitante com parâmetro inicial 0
pv_casa = 0
p_empate = 0
pv_fora = 0

#Modelo de Poisson em prática para calcaular as probabilidades
for gols_casa in range(8):
    for gols_fora in range(8):
        probabilidade_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
        if gols_casa == gols_fora:
            p_empate += probabilidade_resultado
        elif gols_casa > gols_fora:
            pv_casa += probabilidade_resultado
        elif gols_casa < gols_fora:
            pv_fora += probabilidade_resultado

# LInha para tirar possíveis alertas
pd.options.mode.chained_assignment = None  # default='warn'

# Resultado das probabilidades
print()
print("{:.2f}%".format(pv_casa*100)+" vitória do "+mandante)
print("{:.2f}%".format(p_empate*100)+" empate")
print("{:.2f}%".format(pv_fora*100)+" vitória do "+visitante)
